In [12]:
import pandas as pd
from bs4 import BeautifulSoup
from dataclasses import dataclass
import re

@dataclass
class StudyDetails:
    nct: str
    total_participants: int
    num_female: int
    num_male: int

studies = pd.read_stata('esophageal_scraped_tables.dta')
ctg_studies = pd.read_excel('Esophageal Cancer Studies 2.xlsx', sheet_name=0, skiprows=1)
# studies that are alos in ctg_studies
studies = studies[studies['nct'].isin(ctg_studies['NCT Number'])]
studies = studies.sort_values(by=['nct'])

def modify_table_strings(table: str) -> str:
    table = table.replace('<!-- -->', ' ')
    table = table.replace('\n   ', ' ')
    table = table.replace('\n', ' ')
    return table

# apply modify_table_strings to each row in studies['table']
studies['table'] = studies['table'].apply(modify_table_strings)
# Drop "Total Number", "Female", and "Male" from ctg_studies

In [13]:
def get_sex_breakdown(row):
    # Iterate through each tag in the first element of soup
    # first tag in soup.children
    nct = row[0]
    table = row[1]
    num_female, num_male, total_participants = -1, -1, -1
    try:
        soup = BeautifulSoup(table, 'html.parser')
        soup_table = soup.select_one('table')
        
        # get the columns in the soup_table
        cols = soup_table.find_all('th')
        # check the .text of the cols for "Total", case insensitive
        total_col = None
        for idx, col in enumerate(cols):
            if "Total" in col.text or "total" in col.text:
                total_col = idx
                break

        # Find the element that has the text "Age" in it.
        # Then find the root parent
        tds = soup_table.find_all('td')
        relevant_tbody = ""
        for td in tds:
            if ('Sex' in td.text) or ("Gender" in td.text):
                relevant_tbody = td.parent.find_next("tbody")      
        if relevant_tbody == "": return (nct, -1, -1, -1)      

        sex_soup = BeautifulSoup(str(relevant_tbody), 'html.parser')

        trs = sex_soup.find_all('tr')
        if len(cols) == 2:
            # Continue as normal
            for tr in trs:
                tds = tr.find_all('td')
                if "Female" in tds[0].text:
                    if "%" in tds[-1].text:
                        # extract the first digit using regex
                        num_female = int(re.match(r'\s*([^\d]+)\s*(\d+)', tds[-1].text).group(2))
                    else: num_female = ''.join(filter(str.isdigit, tds[-1].text))
                    if num_female == "": num_female = -1
                    else: num_female = int(num_female)
                elif "Male" in tds[0].text:
                    if "%" in tds[-1].text:
                        # extract the first digit using regex
                        num_male = int(re.match(r'\s*([^\d]+)\s*(\d+)', tds[-1].text).group(2))
                    else: num_male = ''.join(filter(str.isdigit, tds[-1].text))
                    if num_male == "": num_male = -1
                    else: num_male = int(num_male)
                elif "Number Analyzed" in tds[0].text:
                    total_participants = ''.join(filter(str.isdigit, tds[-1].text))
                    if total_participants == "": total_participants = -1
                    else: total_participants = int(total_participants)
        elif len(cols) == 3 and total_col is not None:
            # Use total col
            print("col == 3 and total_col is not None: ", nct)
            pass
        elif len(cols) == 3 and total_col is None:
            # combine the two columns
            print("col == 3 and total_col is None: ", nct)
            pass
        elif len(cols) > 3 and total_col is not None:
            # use the total column
            # For each tr in trs, get the td at the index of total_col
            for tr in trs:
                tds = tr.find_all('td')
                if "Female" in tds[0].text:
                    if "%" in tds[total_col].text:
                        # extract the first digit using regex
                        num_female = int(re.match(r'\s*([^\d]+)\s*(\d+)', tds[total_col].text).group(2))
                    else: num_female = ''.join(filter(str.isdigit, tds[total_col].text))
                    if num_female == "": num_female = -1
                    else: num_female = int(num_female)
                elif "Male" in tds[0].text:
                    if "%" in tds[total_col].text:
                        # extract the first digit using regex
                        num_male = int(re.match(r'\s*([^\d]+)\s*(\d+)', tds[total_col].text).group(2))
                    else: num_male = ''.join(filter(str.isdigit, tds[total_col].text))
                    if num_male == "": num_male = -1
                    else: num_male = int(num_male)
                elif "Number Analyzed" in tds[0].text:
                    total_participants = ''.join(filter(str.isdigit, tds[total_col].text))
                    if total_participants == "": total_participants = -1
                    else: total_participants = int(total_participants)
        elif len(cols) > 3 and total_col is None:
            # need to manually check
            print("col > 3: Gender Breakdown manually check: ", nct)
            pass
    except Exception as e:
        print("Sex Exception: ", nct)
        return (nct, -1, -1, -1)
    return (nct, num_female, num_male, total_participants)

def get_age_breakdown(row):
    nct = row[0]
    table = row[1]
    mean, median = -1, -1
    mean_flag, median_flag = False, False

    if "Age, Continuous" not in table: return (nct, -1, -1)
    try:
        soup = BeautifulSoup(table, 'html.parser')
        soup_table = soup.select_one('table')
        # get the columns in the soup_table
        cols = soup_table.find_all('th')
        # check the .text of the cols for "Total", case insensitive
        total_col = None
        for idx, col in enumerate(cols):
            if "Total" in col.text or "total" in col.text:
                total_col = idx
                break

        # Find the element that has the text "Age" in it.
        # Then find the root parent
        tds = soup_table.find_all('td')
        relevant_tbody = ""
        for td in tds:
            if ('Age, Continuous' in td.text):
                if ("Mean") in td.text: mean_flag = True
                if ("Median") in td.text: median_flag = True
                relevant_tbody = td.parent.find_next("tbody")  
        if relevant_tbody == "": 
            print(f"Could not find relevant_tbody for {nct}")
            return (nct, -1, -1, -1)      

        age_soup = BeautifulSoup(str(relevant_tbody), 'html.parser')
        trs = age_soup.find_all('tr')
        tr =  trs[-1]
        
        if len(cols) == 2:
            # Continue as normal
            tds = tr.find_all('td')

            if mean_flag: mean = float(re.match(r'\s*([0-9]+(?:\.\d+)?)\s*', tds[-1].text).group(1))
            if median_flag: median = float(re.match(r'\s*([0-9]+(?:\.\d+)?)\s*', tds[-1].text).group(1))
        elif len(cols) == 3 and total_col is not None:
            # Use total col
            print("col == 3 and total_col is not None: ", nct)
            pass
        elif len(cols) == 3 and total_col is None:
            # combine the two columns
            print("col == 3 and total_col is None: ", nct)
            pass
        elif len(cols) > 3 and total_col is not None:
            tds = tr.find_all('td')
            # print(nct)
            if mean_flag: mean = float(re.match(r'\s*([0-9]+(?:\.\d+)?)\s*', tds[total_col].text).group(1))
            # print(tds[total_col].text)
            if median_flag: median = float(re.match(r'\s*([0-9]+(?:\.\d+)?)\s*', tds[total_col].text).group(1))
        elif len(cols) > 3 and total_col is None:
            # need to manually check
            print("col > 3: Gender Breakdown manually check: ", nct)
            pass
    except Exception as e:
        print("Age Exception: ", nct)
        return (nct, -1, -1)
    return (nct, mean, median)

@dataclass
class RaceEthnicity:
    american_indian: int
    asian: int
    native_hawaiian: int
    black: int
    white: int
    mixed: int
    race_unknown: int
    hispanic: int
    not_hispanic: int
    ethnicity_unknown: int
    race_ethnicity_flag: int

def get_race_breakdown(row):
    # Flag if "Race/Ethnicity" is in the table
    race_ethnicity_flag = False
    nct = row[0]
    table = row[1]

    # race categories
    american_indian, asian, native_hawaiian, black, white, mixed, race_unknown = -1, -1, -1, -1, -1, -1, -1

    # nih ethnicity
    hispanic, not_hispanic, ethnicity_unknown = -1, -1, -1

    if "Race" in table or "Ethnicity" in table: race_ethnicity_flag = True
    else: return (nct, RaceEthnicity(-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1))

    try:
        soup = BeautifulSoup(table, 'html.parser')
        soup_table = soup.select_one('table')
        # get the columns in the soup_table
        cols = soup_table.find_all('th')
        # check the .text of the cols for "Total", case insensitive
        total_col = None
        for idx, col in enumerate(cols):
            if "Total" in col.text or "total" in col.text:
                total_col = idx
                break

        tds = soup_table.find_all('td')
        ethnicity_tbody, race_tbody = "", ""
        for td in tds:
            if ('Ethnicity (NIH/OMB)' in td.text):
                ethnicity_tbody = td.parent.find_next("tbody")  
            if ("Race (NIH/OMB)" in td.text):
                race_tbody = td.parent.find_next("tbody")

        # Ethnicity
        ethnicity_soup = BeautifulSoup(str(ethnicity_tbody), 'html.parser')
        trs = ethnicity_soup.find_all('tr')
        if len(cols) == 2:
            for tr in trs:
                tds = tr.find_all('td')
                if "Not Hispanic or Latino" in tds[0].text:
                    not_hispanic = int(re.match(r'\s*([^\d]+)\s*(\d+)', tds[-1].text).group(2))
                    # print("Not Hispanic or Latino: ", not_hispanic)
                elif "Hispanic or Latino" in tds[0].text:
                    hispanic = int(re.match(r'\s*([^\d]+)\s*(\d+)', tds[-1].text).group(2))
                    # print("Hispanic or Latino: ", hispanic)
                elif "Unknown or Not Reported" in tds[0].text:
                    ethnicity_unknown = int(re.match(r'\s*([^\d]+)\s*(\d+)', tds[-1].text).group(2))
                    # print("Unknown ethnicity:", ethnicity_unknown)
        else:
            for tr in trs:
                tds = tr.find_all('td')
                if "Not Hispanic or Latino" in tds[0].text:
                    not_hispanic = int(re.match(r'\s*([^\d]+)\s*(\d+)', tds[total_col].text).group(2))
                    # print("Not Hispanic or Latino: ", not_hispanic)
                elif "Hispanic or Latino" in tds[0].text:
                    hispanic = int(re.match(r'\s*([^\d]+)\s*(\d+)', tds[total_col].text).group(2))
                    # print("Hispanic or Latino: ", hispanic)
                elif "Unknown or Not Reported" in tds[0].text:
                    ethnicity_unknown = int(re.match(r'\s*([^\d]+)\s*(\d+)', tds[total_col].text).group(2))
                    # print("Unknown ethnicity:", ethnicity_unknown)

        # Race
        race_soup = BeautifulSoup(str(race_tbody), 'html.parser')
        trs = race_soup.find_all('tr')
        if len(cols) == 2:
            for tr in trs:
                tds = tr.find_all('td')
                if "American Indian or Alaska Native" in tds[0].text:
                    american_indian = int(re.match(r'\s*([^\d]+)\s*(\d+)', tds[-1].text).group(2))
                elif "Asian" in tds[0].text:
                    asian = int(re.match(r'\s*([^\d]+)\s*(\d+)', tds[-1].text).group(2))
                    # print("Asian:", asian)
                elif "Native Hawaiian or Other Pacific Islander" in tds[0].text:
                    native_hawaiian = int(re.match(r'\s*([^\d]+)\s*(\d+)', tds[-1].text).group(2))
                    # print("Native Hawaiian or Other Pacific Islander:", native_hawaiian)
                elif "Black or African American" in tds[0].text:
                    black = int(re.match(r'\s*([^\d]+)\s*(\d+)', tds[-1].text).group(2))
                    # print("Black or African American:", black)
                elif "White" in tds[0].text:
                    white = int(re.match(r'\s*([^\d]+)\s*(\d+)', tds[-1].text).group(2))
                    # print("White:", white)
                elif "Unknown or Not Reported" in tds[0].text:
                    race_unknown = int(re.match(r'\s*([^\d]+)\s*(\d+)', tds[-1].text).group(2))
                    # print("Race unknown:", race_unknown)
                elif "More than one race" in tds[0].text:
                    mixed = int(re.match(r'\s*([^\d]+)\s*(\d+)', tds[-1].text).group(2))
                    # print("Mixed:", mixed)
        else:
            for tr in trs:
                tds = tr.find_all('td')
                if "American Indian or Alaska Native" in tds[0].text:
                    american_indian = int(re.match(r'\s*([^\d]+)\s*(\d+)', tds[total_col].text).group(2))
                elif "Asian" in tds[0].text:
                    asian = int(re.match(r'\s*([^\d]+)\s*(\d+)', tds[total_col].text).group(2))
                    # print("Asian:", asian)
                elif "Native Hawaiian or Other Pacific Islander" in tds[0].text:
                    native_hawaiian = int(re.match(r'\s*([^\d]+)\s*(\d+)', tds[total_col].text).group(2))
                    # print("Native Hawaiian or Other Pacific Islander:", native_hawaiian)
                elif "Black or African American" in tds[0].text:
                    black = int(re.match(r'\s*([^\d]+)\s*(\d+)', tds[total_col].text).group(2))
                    # print("Black or African American:", black)
                elif "White" in tds[0].text:
                    white = int(re.match(r'\s*([^\d]+)\s*(\d+)', tds[total_col].text).group(2))
                    # print("White:", white)
                elif "Unknown or Not Reported" in tds[0].text:
                    race_unknown = int(re.match(r'\s*([^\d]+)\s*(\d+)', tds[total_col].text).group(2))
                    # print("Race unknown:", race_unknown)
                elif "More than one race" in tds[0].text:
                    mixed = int(re.match(r'\s*([^\d]+)\s*(\d+)', tds[total_col].text).group(2))
                    # print("Mixed:", mixed)

    except Exception as e:
        print(f"Race/Ethnicity Exception: {nct}")
        return (nct, RaceEthnicity(-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1))

    ret_object = RaceEthnicity(american_indian=american_indian, \
                            asian=asian,\
                            native_hawaiian=native_hawaiian,\
                            black=black,\
                            white=white,\
                            race_unknown=race_unknown,\
                            mixed=mixed,\
                            hispanic=hispanic,\
                            not_hispanic=not_hispanic,\
                            ethnicity_unknown=ethnicity_unknown,\
                            race_ethnicity_flag=race_ethnicity_flag)
    
    return (nct, ret_object)

In [14]:
study_details = []
ncts_tables = list(zip(studies['nct'], studies['table']))

gender_breakdowns = [get_sex_breakdown(row) for row in ncts_tables]
age_breakdowns = [get_age_breakdown(row) for row in ncts_tables]
race_ethnicity_breakdowns = [get_race_breakdown(row) for row in ncts_tables]

Age Exception:  NCT00493025
Age Exception:  NCT00526669
Age Exception:  NCT02213133
Age Exception:  NCT02743494
Age Exception:  NCT03386721
Age Exception:  NCT03449030
Age Exception:  NCT04208958
Age Exception:  NCT04225026
Age Exception:  NCT04424641


In [15]:
gender_breakdowns_df = pd.DataFrame(gender_breakdowns, columns=["NCT Number", "Female", "Male", "Total Number"])
age_breakdowns_df = pd.DataFrame(age_breakdowns, columns=["NCT Number", "Mean", "Median"])
race_ethnicity_breakdowns_flattened = [(nct, data.american_indian, data.asian, data.native_hawaiian, 
                              data.black, data.white, data.mixed, data.race_unknown, data.hispanic, data.not_hispanic, 
                              data.ethnicity_unknown, data.race_ethnicity_flag) for nct, data in race_ethnicity_breakdowns]
race_ethnicity_breakdowns_df = pd.DataFrame(race_ethnicity_breakdowns_flattened, columns=["NCT Number", "Native American", "Asian", "Pacific", 
                                                                                "Black", "White", "Mixed", "Unknown Race", "Hispanic",
                                                                                "Non-His", "Unknown Ethnicity", "Race/Ethnicity Flag"])
new_ctg_studies = ctg_studies.merge(gender_breakdowns_df, on="NCT Number", how="left")
new_ctg_studies = new_ctg_studies.merge(age_breakdowns_df, on="NCT Number", how="left")
new_ctg_studies = new_ctg_studies.merge(race_ethnicity_breakdowns_df, on="NCT Number", how="left")

# new_ctg_studies.to_excel('lung_cancer_studies_scraped.xlsx', index=False)

In [16]:
# start_dates = pd.read_csv("ctg-studies (6).csv")
# # merge with new_ctg_studies on NCT Number
# new_ctg_studies = new_ctg_studies.merge(start_dates, on="NCT Number", how="left")
new_ctg_studies.to_excel('esophageal_cancer_studies_scraped.xlsx', index=False)